In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel, GPT2LMHeadModel
from google.colab import drive
import os
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [6]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [7]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(0))
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
<s>
</s>
<usr>
<pad>
<sys>


In [8]:
import pandas as pd
import tqdm
import urllib.request

In [9]:
df = pd.read_csv('/content/drive/MyDrive/chatbot/dialog_chatbot.csv')
len(df)

1105919

In [10]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(df.loc[df['Q'].str.contains('\*')].index, axis=0, inplace=True)
df.drop(df.loc[df['A'].str.contains('\*')].index, axis=0, inplace=True)
df.drop(df.loc[df['Q'].str.len() >= 50].index, axis=0, inplace=True)
df.drop(df.loc[df['A'].str.len() >= 50].index, axis=0, inplace=True)

len(df)

1046263

In [11]:
intet_label = list(df['Q_intent'].unique())

label_dict = {}

for idx, intent_lab in enumerate(intet_label) :
    label_dict[idx] = intent_lab

label_dict

{0: '(단언) 진술하기',
 1: '(지시) 충고/제안하기',
 2: '(단언) 주장하기',
 3: '(지시) 질문하기',
 4: '(지시) 부탁하기',
 5: '(단언) 반박하기',
 6: '(표현) 감사하기',
 7: '(표현) 사과하기',
 8: '(지시) 명령/요구하기',
 9: '턴토크 사인(관습적 반응)',
 10: '(언약) 약속하기(제3자와)/(개인적 수준)',
 11: '(표현) 긍정감정 표현하기',
 12: '(표현) 부정감정 표현하기',
 13: '(언약) 거절하기',
 14: '(언약) 위협하기',
 15: '(표현) 인사하기',
 16: '(선언/위임하기)'}

In [12]:
Q_int_li = list(set(df['Q_intent'].unique()))
A_int_li = list(set(df['A_intent'].unique()))

for i in Q_int_li :
  print('Q_intent',i,df.loc[df['Q_intent'] == i, 'Q_intent'].count())
print('='*50)
for i in A_int_li :
  print('A_intent',i,df.loc[df['A_intent'] == i, 'A_intent'].count())

Q_intent (선언/위임하기) 64
Q_intent (표현) 인사하기 312
Q_intent 턴토크 사인(관습적 반응) 3490
Q_intent (표현) 부정감정 표현하기 6139
Q_intent (언약) 약속하기(제3자와)/(개인적 수준) 12681
Q_intent (표현) 긍정감정 표현하기 4784
Q_intent (단언) 진술하기 325169
Q_intent (언약) 위협하기 64
Q_intent (지시) 부탁하기 5981
Q_intent (표현) 사과하기 501
Q_intent (지시) 명령/요구하기 4512
Q_intent (단언) 주장하기 311492
Q_intent (언약) 거절하기 406
Q_intent (지시) 질문하기 328664
Q_intent (단언) 반박하기 3087
Q_intent (표현) 감사하기 1266
Q_intent (지시) 충고/제안하기 37651
A_intent (선언/위임하기) 142
A_intent (표현) 인사하기 586
A_intent 턴토크 사인(관습적 반응) 10881
A_intent (표현) 부정감정 표현하기 6981
A_intent (언약) 약속하기(제3자와)/(개인적 수준) 15984
A_intent (표현) 긍정감정 표현하기 6921
A_intent (단언) 진술하기 310380
A_intent (언약) 위협하기 83
A_intent (지시) 부탁하기 5825
A_intent (표현) 사과하기 973
A_intent (지시) 명령/요구하기 4584
A_intent (단언) 주장하기 373022
A_intent (언약) 거절하기 598
A_intent (지시) 질문하기 262220
A_intent (단언) 반박하기 4140
A_intent (표현) 감사하기 2304
A_intent (지시) 충고/제안하기 40639


In [13]:
train_data = pd.DataFrame()

for v,k in label_dict.items():
    len_ = df.loc[df['Q_intent'] == k,'Q_intent'].count()
    if len_ > 10000 :
        train_data = pd.concat([train_data,df.loc[df['Q_intent'] == k].sample(n=10000)],ignore_index=True)
    else :
        train_data = pd.concat([train_data,df.loc[df['Q_intent'] == k]],ignore_index=True)

for v,k in label_dict.items():
    len_ = df.loc[df['A_intent'] == k,'A_intent'].count()
    if len_ > 10000 :
        train_data = pd.concat([train_data,df.loc[df['A_intent'] == k].sample(n=10000)],ignore_index=True)
    else :
        train_data = pd.concat([train_data,df.loc[df['A_intent'] == k]],ignore_index=True)

train_data.drop(train_data[train_data.duplicated()].index, axis=0, inplace=True)
train_data = train_data.sample(frac=1).reset_index(drop=True)
train_data

,Q,Q_intent,A,A_intent
0,남원은 또 왜,(지시) 질문하기,광한루원 야경 보니까 엄청 이쁘더라고~,(단언) 진술하기
1,오 수능 때 물리를 선택했어?,(지시) 질문하기,웅웅 난 물리가 좋았거든,(단언) 진술하기
2,너희 강아지 키우면 내가 산책 시켜 줄게,(지시) 충고/제안하기,나만 없어 또 ㅠㅠ,턴토크 사인(관습적 반응)
3,그렇죠 월세는 진짜 돈 아깝죠 키키,(단언) 주장하기,요즘 유튜브에 부동산 정보 많더라,(단언) 진술하기
4,나는 옷도 팔아보고 엄마 아빠가 농사해서 히터 이런 거 사보고 했어 키키,(단언) 진술하기,대단하다 키키,(단언) 주장하기
...,...,...,...,...
164834,우리 준호 제대했으니까 빨리 찍어,(지시) 충고/제안하기,이게 군대 가기전 마지막이래,(단언) 진술하기
164835,누룽지 근데 맛있더라 꼭 먹어봐,(지시) 충고/제안하기,그래 라쿤 패딩 꺼내야지 폭풍 스팀질,(단언) 주장하기
164836,웅~ 하고 가다가 팍팍팍팍 튀는소리,(단언) 진술하기,하하 어후 그건 안되겠다 여보,(단언) 반박하기
164837,왜 일본이야 키키 설마 디즈니 때문에?,(지시) 질문하기,응응 디즈니… 나 혈중 테마파크 농도가 너무 떨어졌어…,(단언) 주장하기


In [14]:
batch_size = 128

In [15]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
    yield bos_token + sent + eos_token

In [16]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [17]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [18]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    1     2  9112 ...     3     3     3]
 [    1     2  9114 ...     3     3     3]
 [    1     2 10099 ...     3     3     3]
 ...
 [    1     2  9063 ...     3     3     3]
 [    1     2 35704 ...     3     3     3]
 [    1     2 15247 ...     3     3     3]], shape=(128, 44), dtype=int32)


In [19]:
tokenizer.decode(batch[0])

'</s><usr> 남원은 또 왜<sys> 광한루원 야경 보니까 엄청 이쁘더라고~</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [20]:
print(batch[0])

tf.Tensor(
[    1     2  9112 10116  9108 10401     4 45621  8104  9893  6884 17272
  6969 15546  9018  7748  7208  9329   468     1     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3], shape=(44,), dtype=int32)


In [21]:
print(tokenizer.encode('</s><usr>12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'))

[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [22]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [23]:
steps = len(train_data) // batch_size + 1
print(steps)

1288


In [24]:
EPOCHS = 5

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-24-b887b383bcd0>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/1288 [00:00<?, ?it/s]

[Epoch:    1] cost = 1.98342824


  0%|          | 0/1288 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.8155843


...

[Epoch:    3] cost = 1.68882394


  0%|          | 0/1288 [00:00<?, ?it/s]

[Epoch:    4] cost = 1.54227042


  0%|          | 0/1288 [00:00<?, ?it/s]

[Epoch:    5] cost = 1.39086246


In [25]:
text = '키키 난 이미 찌들대로 찌들었나 보다'

In [26]:
sent = '<usr>' + text + '<sys>'

In [27]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
print('tokenizer.encode(sent) :',tokenizer.encode(sent))
print('input_ids:',input_ids)
input_ids = tf.convert_to_tensor([input_ids])
print('input_ids:',input_ids)

tokenizer.encode(sent) : [2, 10252, 8511, 9769, 10074, 18328, 7285, 9506, 18328, 7285, 8017, 7055, 10903, 4]
input_ids: [1, 2, 10252, 8511, 9769, 10074, 18328, 7285, 9506, 18328, 7285, 8017, 7055, 10903, 4]
input_ids: tf.Tensor(
[[    1     2 10252  8511  9769 10074 18328  7285  9506 18328  7285  8017
   7055 10903     4]], shape=(1, 15), dtype=int32)


In [28]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)
print(output)

tf.Tensor(
[[    1     2 10252  8511  9769 10074 18328  7285  9506 18328  7285  8017
   7055 10903     4 10252  8511  9063  9782  9148 10348  9443 11630  8711
   7661 10252  8511     1]], shape=(1, 28), dtype=int32)


In [29]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print('decoded_sentence :',decoded_sentence)

decoded_sentence : </s><usr> 키키 난 이미 찌들대로 찌들었나 보다<sys> 키키 나 지금부터라도 잘 준비해봐 키키</s>


In [30]:
# decoded_sentence.split('<sys> ')[1].replace('</s>', '')

In [31]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
print('output :',output)
tokenizer.decode(output[0].numpy().tolist())
print('tokenizer.decode(output[0].numpy().tolist()) :',tokenizer.decode(output[0].numpy().tolist()))

output : tf.Tensor(
[[    1     2 10252  8511  9769 10074 18328  7285  9506 18328  7285  8017
   7055 10903     4 10252  8511  9063  9022  6853   739  7662  8006     1]], shape=(1, 24), dtype=int32)
tokenizer.decode(output[0].numpy().tolist()) : </s><usr> 키키 난 이미 찌들대로 찌들었나 보다<sys> 키키 나 그거 봤어</s>


In [32]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  print('sent :',sent)
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  print('tokenizer.encode(sent) :',tokenizer.encode(sent))
  print('[tokenizer.bos_token_id] :',[tokenizer.bos_token_id])
  print('input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent) :',input_ids)
  input_ids = tf.convert_to_tensor([input_ids])
  print('input_ids = tf.convert_to_tensor([input_ids]) :',tf.convert_to_tensor([input_ids]))
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  print('output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20) :',output)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  print('output[0] :',output[0])
  print('output[0].numpy() :',output[0].numpy())
  print('output[0].numpy().tolist() :',output[0].numpy().tolist())
  print('sentence = tokenizer.decode(output[0].numpy().tolist()) :',tokenizer.decode(output[0].numpy().tolist()))
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  print("sentence.split('<sys> ') :",sentence.split('<sys> '))
  print("sentence.split('<sys> ')[1] :",sentence.split('<sys> ')[1])
  print("chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '') :",sentence.split('<sys> ')[1].replace('</s>', ''))
  return chatbot_response

In [33]:
def chat():
    while 1:
        sen = input("user > ").strip()
        if sen == '끝':
            break
        respon = return_answer_by_chatbot(sen)
        print("Chatbot > {}".format(respon.strip()))

In [34]:
chat()

user > 안녕 
sent : <usr>안녕<sys>
tokenizer.encode(sent) : [2, 25906, 4]
[tokenizer.bos_token_id] : [1]
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent) : [1, 2, 25906, 4]
input_ids = tf.convert_to_tensor([input_ids]) : tf.Tensor([[[    1     2 25906     4]]], shape=(1, 1, 4), dtype=int32)
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20) : tf.Tensor([[    1     2 25906     4  9320     1]], shape=(1, 6), dtype=int32)
output[0] : tf.Tensor([    1     2 25906     4  9320     1], shape=(6,), dtype=int32)
output[0].numpy() : [    1     2 25906     4  9320     1]
output[0].numpy().tolist() : [1, 2, 25906, 4, 9320, 1]
sentence = tokenizer.decode(output[0].numpy().tolist()) : </s><usr> 안녕<sys> 아니</s>
sentence.split('<sys> ') : ['</s><usr> 안녕', '아니</s>']
sentence.split('<sys> ')[1] : 아니</s>
chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '') : 아니
Chatbot > 아니
user > 만나서 반가워
sent : <usr>만나서 반가워<sys>
tokenizer.encode(sent) : [2, 11667, 7788, 362

In [35]:
MODEL_NAME = 'koGPT_Chatbot'
MODEL_SAVE_PATH = os.path.join("/content/drive/MyDrive/chatbot/koGPT_Chatbot", MODEL_NAME) # change this to your preferred location
print(MODEL_SAVE_PATH)
print('='*50)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")

else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot
/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot -- Folder create complete 



('/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot/tokenizer_config.json',
 '/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot/special_tokens_map.json',
 '/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot/vocab.json',
 '/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot/merges.txt',
 '/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot/added_tokens.json',
 '/content/drive/MyDrive/chatbot/koGPT_Chatbot/koGPT_Chatbot/tokenizer.json')